## Boletin semanal asociación de productores independientes de banano

##### Este notebook tiene como objetivo construir la tabla y graficos de reporte que realiza ICC

In [1]:
## Libreías generales
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

#comando para visualizar todas las columnas del df
pd.pandas.set_option('display.max_columns', None)

In [2]:
### Lectura de archivo diario 

dfs_diario=pd.read_parquet("diario_complete.parquet")
dfs_diario.head()

,estacion,date,semana,dia,mes,año,temperatura_min_diaria,temperatura_max_diaria,temperatura_promedio_diaria,radiacion_diaria_acumulada,radiacion_diaria_promedio,humedad_relativa_min_diaria,humedad_relativa_max_diaria,humedad_relativa_media_diaria,lluvia_diaria,velocidad_viento_media_diario,velocidad_viento_max_diaria,amplitud_termica,radiacion_global,radiacion_dia_despejado,N Daylight hours,Ra,estacion_resume,altitud,estrato,latitud,longitud,latitud_rad,radiacion_media_estimada Heargreaves,Rg,n,dia_juliano,dr,declinacion,ang_horario_puesta_sol,heliofania,ra_MJm-2day-1,radiacion_solar_piranometro,constante_psicometrica,pendiente_curva_presion_satu_vapor,presion_saturacion_vapor,presion_real_vapor,deficit_presion_vapor,rns,rnl,rn,velocidad_viento_altura_standar,FAO-Penman-Montieth,deficit
0,CENCEN,2008-06-11,24,11,6,2008,19.4,31.9,25.094792,24413.6,254.308333,52.0,93.0,76.666667,2.6,4.514583,21.1,12.5,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,250.542075,21.894361,7.737019,163,0.968842,0.404105,1.680265,12.836249,38.200705,21.97224,0.065039,0.189613,3.490402,2.276839,1.213562,16.918625,3.416555,13.502069,0.937969,4.604556,-2.004556
1,CENCEN,2009-06-11,24,11,6,2009,21.2,32.7,25.609375,26112.4,272.004167,58.0,96.0,81.541667,9.4,7.118750,29.6,11.5,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,240.311516,23.417861,8.561034,162,0.969033,0.402959,1.679917,12.833590,38.208086,23.50116,0.065039,0.194734,3.731921,2.642881,1.089039,18.095893,3.352965,14.742928,1.479021,5.084424,4.315576
2,CENCEN,2010-06-11,23,11,6,2010,22.1,32.5,26.316667,22499.4,234.368750,53.0,95.0,78.062500,1.2,3.218750,20.0,10.4,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,228.529533,20.177687,6.808524,162,0.969033,0.402959,1.679917,12.833590,38.208086,20.24946,0.065039,0.201963,3.775434,2.559599,1.215835,15.592084,2.769762,12.822322,0.668741,4.313769,-3.113769
3,CENCEN,2011-06-11,23,11,6,2011,22.1,32.7,26.965625,19675.1,204.948958,57.0,96.0,78.562500,4.8,2.257292,10.1,10.6,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,230.716469,17.644826,5.438578,162,0.969033,0.402959,1.679917,12.833590,38.208086,17.70759,0.065039,0.208793,3.803104,2.686487,1.116617,13.634844,2.116141,11.518703,0.468984,3.812202,0.987798
4,CENCEN,2012-06-11,24,11,6,2012,22.2,32.5,26.411458,17912.7,186.590625,53.0,91.0,76.812500,0.0,2.130208,21.1,10.3,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,227.428180,16.064288,4.583714,163,0.968842,0.404105,1.680265,12.836249,38.200705,16.12143,0.065039,0.202949,3.783556,2.513788,1.269768,12.413501,1.908163,10.505338,0.442581,3.527102,-3.527102


In [3]:
##3Lectura archivo de la información por estación

info_estaciones = pd.read_excel('prueba_info.xlsx')

info_estaciones.head()

,No.,Estacion,Codigo,Tipo,Resolución\ntemporal *,Fecha de inicio,Años,Longitud,Latitud,Altitud\n(msnm),Estrato,t,LUGAR,RESPONSABLE,Temperatura,Radiación,Humedad Relativa,Precipitación pluvial,Mojadura\nde la hoja,Velocidad y dirección del viento,Presión\natmosférica,Brillo\nsolar
0,1,Cengicaña,CEN-CEN,Automática GPRS,15 minutos,1997-11-19,24.931507,-91.055468,14.330962,297,Alto,Alto,Santa Lucía Cotz. Escuintla,ICC,Si,Si,Si,Si,Si,Si,No,Si
1,2,Tehuantepec,LUT-TEH,Automática GPRS,15 minutos,1998-03-04,24.643836,-91.103443,14.168625,67,Bajo,Bajo,Santa Lucia Cotz. Escuintla,La Unión,Si,Si,Si,Si,Si,Si,No,No
2,3,El Bálsamo,PAN-BAL,Automática GPRS,15 minutos,2002-02-13,20.693151,-91.003744,14.281468,280,Medio,Medio,Santa Lucía Cotz. Escuintla,Ingenio Pantaleón,Si,Si,Si,Si,Si,Si,No,No
3,4,Puyumate,TBU-PUY,Automática GPRS,15 minutos,2002-02-14,20.690411,-91.259910,14.261557,77,Bajo,Bajo,"Nueva Concepción, Escuintla",Ingenio Pantaleón,Si,Si,Si,Si,Si,Si,No,No
4,5,San Antonio el Valle,MAG-SAV,Automática GPRS,15 minutos,2002-02-27,20.654795,-91.200961,13.995364,10,Litoral,Litoral,"La Gomera, Escuintla",Ingenio Magdalena,Si,Si,Si,Si,Si,Si,No,No


In [4]:
###Limpieza de las columna de codigo
info_estaciones.Codigo =info_estaciones.Codigo.replace(r'\W+', '', regex=True)
info_estaciones.Codigo = info_estaciones.Codigo.replace(r'\s+', ' ', regex=True)
info_estaciones.Codigo = info_estaciones.Codigo.replace("", np.nan, regex=True)
info_estaciones.Codigo= info_estaciones.Codigo.replace(" ", np.nan, regex=True)

### Columnas que se necesitan
info_estrato =info_estaciones[['Estacion','Codigo','t','Altitud\n(msnm)','LUGAR']]
info_estrato.columns = ['nombre_estacion','estacion','estrato','Altitud\n(msnm)','Ubicacion']

In [5]:
df_semanal_estacion = dfs_diario.groupby(['estacion','semana','año']).agg(
    temp_min_semanal=('temperatura_min_diaria', 'mean'),
    temp_max_semanal=('temperatura_max_diaria', 'mean'),
    temp_promedio_semanal=('temperatura_promedio_diaria', 'mean'),
    rad_diar_prom_semanal=('radiacion_diaria_promedio', 'mean'),
    ampl_term_prom_semanal = ('amplitud_termica', 'mean'),
    hume_rela_med_semanal= ('humedad_relativa_media_diaria','mean'),
    Veloc_viento_prom_max_semanal= ('velocidad_viento_max_diaria', 'mean'),
    Veloc_viento_media_semanal= ('velocidad_viento_media_diario', 'mean' ),
    max_lluvia=('lluvia_diaria', 'max'),
    acumu_lluvia=('lluvia_diaria', 'sum'),
    suma_FAO= ('FAO-Penman-Montieth','sum')

).reset_index().round(2).astype({'semana':int, 'año':int})

df_semanal_estacion

,estacion,semana,año,temp_min_semanal,temp_max_semanal,temp_promedio_semanal,rad_diar_prom_semanal,ampl_term_prom_semanal,hume_rela_med_semanal,Veloc_viento_prom_max_semanal,Veloc_viento_media_semanal,max_lluvia,acumu_lluvia,suma_FAO
0,CENCEN,1,2007,18.90,32.10,24.75,170.07,13.20,72.50,17.90,5.31,0.0,0.0,3.38
1,CENCEN,1,2008,18.62,32.78,24.87,194.77,14.16,63.39,38.88,12.18,0.0,0.0,44.75
2,CENCEN,1,2009,17.25,32.02,23.99,196.95,14.77,70.05,21.15,5.74,0.0,0.0,14.96
3,CENCEN,1,2010,18.34,32.50,24.50,176.98,14.16,70.90,22.46,4.55,24.6,24.6,24.01
4,CENCEN,1,2011,18.14,31.84,24.18,193.61,13.70,74.52,14.49,2.46,0.0,0.0,22.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15660,TULTLA,52,2022,20.00,33.30,25.95,222.19,13.30,83.38,18.30,4.44,0.0,0.0,7.71
15661,TULTLA,53,2015,21.48,33.75,26.99,186.38,12.28,77.79,15.70,6.75,0.0,0.0,15.31
15662,TULTLA,53,2016,22.23,33.23,26.98,186.38,11.00,83.64,18.43,7.05,8.2,8.2,11.26
15663,TULTLA,53,2020,19.70,31.55,24.79,217.36,11.85,84.45,15.05,5.47,4.6,4.6,15.03


In [6]:
df_semanal_estacion['Balance_precipitacion-evapotranspiracion']=df_semanal_estacion.acumu_lluvia-df_semanal_estacion.suma_FAO

In [7]:
df_semanal_estacion=df_semanal_estacion.merge(info_estrato, how='right', on=['estacion'])
df_semanal_estacion.head()

,estacion,semana,año,temp_min_semanal,temp_max_semanal,temp_promedio_semanal,rad_diar_prom_semanal,ampl_term_prom_semanal,hume_rela_med_semanal,Veloc_viento_prom_max_semanal,Veloc_viento_media_semanal,max_lluvia,acumu_lluvia,suma_FAO,Balance_precipitacion-evapotranspiracion,nombre_estacion,estrato,Altitud\n(msnm),Ubicacion
0,CENCEN,1.0,2007.0,18.90,32.10,24.75,170.07,13.20,72.50,17.90,5.31,0.0,0.0,3.38,-3.38,Cengicaña,Alto,297,Santa Lucía Cotz. Escuintla
1,CENCEN,1.0,2008.0,18.62,32.78,24.87,194.77,14.16,63.39,38.88,12.18,0.0,0.0,44.75,-44.75,Cengicaña,Alto,297,Santa Lucía Cotz. Escuintla
2,CENCEN,1.0,2009.0,17.25,32.02,23.99,196.95,14.77,70.05,21.15,5.74,0.0,0.0,14.96,-14.96,Cengicaña,Alto,297,Santa Lucía Cotz. Escuintla
3,CENCEN,1.0,2010.0,18.34,32.50,24.50,176.98,14.16,70.90,22.46,4.55,24.6,24.6,24.01,0.59,Cengicaña,Alto,297,Santa Lucía Cotz. Escuintla
4,CENCEN,1.0,2011.0,18.14,31.84,24.18,193.61,13.70,74.52,14.49,2.46,0.0,0.0,22.22,-22.22,Cengicaña,Alto,297,Santa Lucía Cotz. Escuintla


In [11]:
Boletin_banano=df_semanal_estacion[['nombre_estacion','Altitud\n(msnm)','Ubicacion','semana', 'año','temp_min_semanal','temp_promedio_semanal','temp_max_semanal','hume_rela_med_semanal',
                                    'acumu_lluvia','suma_FAO','Balance_precipitacion-evapotranspiracion']]
Boletin_banano.columns = ['Estación','Altitud(msnm)','Ubicacion','semana', 'año','temp_min_semanal','temp_promedio_semanal','temp_max_semanal','hume_rela_med_semanal',
                                    'Precipitacion acumulada','Evapotranspiración_acumulada','Balance_precipitacion-evapotranspiracion']

In [19]:
"""
se elimnan las estaciones 'PLT':"ICCPLT", 'STA':"ICCSTA", 'PBR':"ICCPBR" y 'YEPO':"ICCYEPO", 
dado que los reportes que realizan desde ICC solo incluyen las estaciones de zonas cañeras, 
por lo tanto se reportan 28 estaciones 

"""

to_remove = Boletin_banano.loc[Boletin_banano.Estación.isin(['Santa Teresa ', 'FCA Yepocapa', 'Central Izalco ', 'Copal ', 'Chaparrastique', 'El Carmen', 'San Clemente', 'Agua Santa', 'Pueblo Real ','El Platanar'])].index

Boletin_banano.drop(to_remove, inplace=True)

In [20]:
Semana= int(input("Digite la semana que desea visualizar"))
periodo=int(input("Digite el año que desea visualizar"))

In [23]:
week_banano=Boletin_banano[(Boletin_banano.semana==Semana)&(Boletin_banano.año==periodo)]
week_banano.head(28)

,Estación,Altitud(msnm),Ubicacion,semana,año,temp_min_semanal,temp_promedio_semanal,temp_max_semanal,hume_rela_med_semanal,Precipitacion acumulada,Evapotranspiración_acumulada,Balance_precipitacion-evapotranspiracion
550,Cengicaña,297,Santa Lucía Cotz. Escuintla,37.0,2021.0,21.11,25.35,31.81,90.55,92.8,29.27,63.53
1326,Tehuantepec,67,Santa Lucia Cotz. Escuintla,37.0,2021.0,22.60,27.30,34.17,82.80,0.2,31.42,-31.22
2102,El Bálsamo,280,Santa Lucía Cotz. Escuintla,37.0,2021.0,21.43,25.66,31.54,86.56,70.4,29.14,41.26
2878,Puyumate,77,"Nueva Concepción, Escuintla",37.0,2021.0,20.87,25.93,34.33,93.88,19.4,29.46,-10.06
3654,San Antonio el Valle,10,"La Gomera, Escuintla",37.0,2021.0,22.31,27.49,34.97,89.67,31.8,31.79,0.01
4393,Amazonas,21,"Masagua, Escuintla",37.0,2021.0,22.40,27.20,33.63,90.99,4.8,29.61,-24.81
5154,Trinidad San Diego,71,"Masagua, Escuintla",37.0,2021.0,22.34,26.99,33.16,87.18,4.8,29.43,-24.63
5930,Irlanda,22,"Tiquisate, Escuintla",37.0,2021.0,24.01,28.60,35.01,91.63,27.2,32.21,-5.01
6706,Bonanza,29,"La Gomera, Escuintla",37.0,2021.0,22.67,27.16,33.74,88.99,47.2,30.23,16.97
7482,Bouganvilia,54,"La Democracia, Escuintla",37.0,2021.0,21.50,27.03,34.41,82.53,1.2,31.95,-30.75


In [ ]:
### Guardar tabla en excel 
week_banano.to_excel('boletin_banano.xlsx')